# Use Lasso to do feature selection first and then do polynomial regression

In [39]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

data = pd.read_csv('../practice/dataset/winequality-red.csv')

# y = df['quality']
# X = df.drop(['quality'], axis=1)
X = data.loc[: , "fixed acidity":"alcohol"]
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [40]:
y = data.loc[:,["quality"]]
y.head()

,quality
0,5
1,5
2,5
3,6
4,5


In [41]:
y.shape

(1599, 1)

In [42]:
## 0. 請把X做normalization 
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

model = linear_model.LinearRegression()
model.fit(X, y)

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X = pd.DataFrame(X)

# y = model.predict(X)

## 1. 建立Lasso模型(假設alpha=0.1)
model = linear_model.Lasso(alpha=0.1)

# Train the model using the training sets
model.fit(X, y)

# The coefficients
print('Coefficients: {}\n'.format(model.coef_))

## 2. 顯示出有多少個係數不為 0
print('number of model coef: {}'.format(np.sum(model.coef_ != 0)))

Coefficients: [ 0.         -0.15459205  0.          0.         -0.         -0.
 -0.         -0.         -0.          0.03926141  0.24947033]

number of model coef: 3


In [43]:
type(X)

pandas.core.frame.DataFrame

In [44]:
print('before feature extraction\nX shape: {}'.format(X.shape))
print(X.head())

before feature extraction
X shape: (1599, 11)
         0         1         2         3         4         5         6   \
0 -0.528360  0.961877 -1.391472 -0.453218 -0.243707 -0.466193 -0.379133   
1 -0.298547  1.967442 -1.391472  0.043416  0.223875  0.872638  0.624363   
2 -0.298547  1.297065 -1.186070 -0.169427  0.096353 -0.083669  0.229047   
3  1.654856 -1.384443  1.484154 -0.453218 -0.264960  0.107592  0.411500   
4 -0.528360  0.961877 -1.391472 -0.453218 -0.243707 -0.466193 -0.379133   

         7         8         9         10  
0  0.558274  1.288643 -0.579207 -0.960246  
1  0.028261 -0.719933  0.128950 -0.584777  
2  0.134264 -0.331177 -0.048089 -0.584777  
3  0.664277 -0.979104 -0.461180 -0.584777  
4  0.558274  1.288643 -0.579207 -0.960246  


In [45]:
mask = model.coef_ != 0
print(mask)

## 3. 使用mask將係數為0對應的特徵剔除
X = X.iloc[:, mask]
print('after feature extraction\nX shape: {}'.format(X.shape))
print(X.head())

[False  True False False False False False False False  True  True]
after feature extraction
X shape: (1599, 3)
         1         9         10
0  0.961877 -0.579207 -0.960246
1  1.967442  0.128950 -0.584777
2  1.297065 -0.048089 -0.584777
3 -1.384443 -0.461180 -0.584777
4  0.961877 -0.579207 -0.960246


In [46]:
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures



## 4. 將抽取過後的特徵做PolynomialFeatures提升維度(假設degree=2)
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_poly, y, test_size=0.3, random_state=1) #random_state 種子值


scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)


model = linear_model.LinearRegression()
model.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

# The coefficients
print('Coefficients: {}\n'.format(model.coef_))
# The mean squared error
print("Mean squared error: {}".format(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('R2 score: {}'.format(r2_score(y_test, y_pred)))

Coefficients: [[ 0.         -0.1979168   0.22973068  0.33944012 -0.02937768 -0.02456795
  -0.0200283  -0.14557193  0.05641426 -0.04929108]]

Mean squared error: 0.40597004977398315
R2 score: 0.3329898562520005
